## 多向量检索器

每个文档存储多个向量通常是有益的。在许多用例中，这是有益的。 LangChain 有一个基础 MultiVectorRetriever ，这使得查询此类设置变得容易。很多复杂性在于如何为每个文档创建多个向量。本笔记本涵盖了创建这些向量和使用 MultiVectorRetriever 的一些常见方法。

为每个文档创建多个向量的方法包括：
- 较小的块：将文档分割成较小的块，然后嵌入这些块（这是 ParentDocumentRetriever）。
- 摘要：为每个文档创建摘要，将其与文档一起嵌入（或代替文档）
- 假设性问题：创建每个文档都适合回答的假设性问题，将这些问题与文档一起嵌入（或代替文档）。

请注意，这还启用了另一种添加嵌入的方法 - 手动。这很棒，因为您可以显式添加导致文档恢复的问题或查询，从而为您提供更多控制权。

In [1]:
from langchain.retrievers.multi_vector import MultiVectorRetriever

In [2]:
from langchain.storage import InMemoryByteStore
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
loaders = [
    TextLoader("./txt/faq-4359.txt",encoding="utf-8"),
    TextLoader("./txt/faq-7923.txt",encoding="utf-8"),
]
loaders

In [6]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

docs

[Document(page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。', metadata={'source': './txt/faq-4359.txt'}),
 Document(page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准。\n\n三、适用范围：华为商城所有产品。\n\n\n\n参与方式：\n\n1、华为商城众测的入口在哪里？\n\n华为商城搜索“众测”，即可看到众测入口，

In [7]:
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000)
# docs = text_splitter.split_documents(docs)

In [8]:
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
embeddings_path = "D:\\ai\\download\\bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_path)

C:\Users\a4154\AppData\Roaming\Python\Python310\site-packages\transformers\utils\hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
d:\ProgramData\anaconda3\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [9]:
# 用于索引子块的向量存储
vectorstore = Chroma(
    collection_name="full_documents", embedding_function=embeddings
)

In [10]:
# 父文档的存储层
store = InMemoryByteStore()
id_key = "doc_id"
# 检索器（空启动）
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)

In [11]:
import uuid

doc_ids = [str(uuid.uuid4()) for _ in docs]
doc_ids

['1177fa05-e477-432c-aa9f-2b070e130c32',
 '8f80d02b-6a27-46ae-ad6a-23cc6e1ec5c8']

In [13]:
from langchain_text_splitters import CharacterTextSplitter
# 用于创建较小块的分割器
child_text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

In [14]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
sub_docs

Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100


[Document(page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '1177fa05-e477-432c-aa9f-2b070e130c32'}),
 Document(page_content='注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”', metadata={'source': './txt/faq-4359.txt', 'doc_id': '1177fa05-e477-432c-aa9f-2b070e130c32'}),
 Document(page_content='二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '1177fa05-e477-432c-aa9f-2b070e130c32'}),
 Document(page_content='三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '1177fa05-e477-432c-aa9f-2b070e130c32'}),
 Document(page_content='四、订单中有多个商品，其中有商品支持0分期利息，为什

In [15]:
#使用一个名为retriever的对象来向一个向量存储（vectorstore）中添加文档，
#并且使用一个文档存储（docstore）来设置文档ID与文档内容之间的映射。
#这两个属性分别用于存储文档的向量化表示和文档的内容。
retriever.vectorstore.add_documents(sub_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [16]:
# Vectorstore 单独检索小块
retriever.vectorstore.similarity_search("众测商品多久发货呢？")[0]

Document(page_content='4、众测商品买下后多久发货？\n\n     您好，请以商品页显示为准。\n\n5、提交活动订单后多久内支付？\n\n     您好，提交订单后最长付款时效为24小时，逾期订单自动取消', metadata={'doc_id': '8f80d02b-6a27-46ae-ad6a-23cc6e1ec5c8', 'source': './txt/faq-7923.txt'})

# 摘要总结
通常，摘要可能能够更准确地提炼出某个块的内容，从而实现更好的检索。在这里，我们展示如何创建摘要，然后嵌入它们。



In [17]:
import uuid

from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [18]:
from langchain_openai import ChatOpenAI, OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://127.0.0.1:1234/v1"
model = ChatOpenAI(
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    temperature=0.3,
)


In [19]:
chain = (
    {"doc": lambda x: x.page_content}
    | ChatPromptTemplate.from_template("总结下面的文档:\n\n{doc}")
    | model
    | StrOutputParser()
)

In [20]:
docs = []
for loader in loaders:
    docs.extend(loader.load())

docs

[Document(page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。\n\n注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”\n\n\n\n二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。\n\n\n\n三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。\n\n\n\n\n\n四、订单中有多个商品，其中有商品支持0分期利息，为什么提交后却没有0分期利息？\n\n0分期利息商品不能和其它商品一起购买，如果和其他商品购买而不能享用0分期利息，建议取消原来的订单，重新购买时把0分期利息商品和其他商品分开单独购买；且0分期利息的分期数是固定的，如6期0分期利息，并不是3/6/12都提供0分期利息的。\n\n\n\n五、小程序是否支持0分期利息？\n\n华为商城小程序暂不支持0分期利息。', metadata={'source': './txt/faq-4359.txt'}),
 Document(page_content='众测活动\n\n整体介绍：\n\n一、活动定义：众测是以低价试销的形式，通过收集评价、销量等方法，用于测试市场对新商品的反应，用于及时优化销售策略和引导商家改进。\n\n二、优势：众测价通常比较优惠，以不高于大促促销价为原则，最终以与物权方谈判结果为准。\n\n三、适用范围：华为商城所有产品。\n\n\n\n参与方式：\n\n1、华为商城众测的入口在哪里？\n\n华为商城搜索“众测”，即可看到众测入口，

In [21]:
summaries = chain.batch(docs, {"max_concurrency": 5})
summaries

['华为商城推出了一项名为“0分期利息”的活动，允许用户使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息，由华为商城承担。从2023年开始，商城将相关宣传调整为“0分期利息”，以符合监管机构关于分期业务规范的要求。目前，该活动仅支持部分单品参与，且不能与其他商品一起购买。用户需要在订单提交成功后在支付方式页面选择分期支付并确认0分期利息的支付方式和期数后完成支付。需要注意的是，华为商城小程序暂不支持0分期利息。',
 '华为商城众测活动是测试市场对新商品反应的一种低价试销形式，通过收集评价和销量来优化销售策略并引导商家改进。众测价通常优惠，不超过大促促销价原则，最终价格以与物权方谈判结果为准。所有产品都适用于华为商城的众测活动。\n\n参与方式：\n1. 在华为商城搜索“众测”进入入口；\n2. 众测频道每周一至周五不定期更新上架，部分商品可能会特别调整；\n3. 一款产品的众测时间通常为10-20天（热销的商品可能会延期5-10天）。\n\n常见问题：\n1. 众测商品主要来源是华为商城上架的新品和爆款商品回归上线；\n2. 众测商品价格对标618和双十一，一般是该商品某段时间的最低促销价；\n3. 众测商品质量与正式上架商品一致；\n4. 请以商品页显示为准；\n5. 提交订单后最长付款时效为24小时，逾期订单自动取消；\n6. 成功下单后可通过华为商城手机APP、华为商城手机WAP版、华为商城电脑网页版查询众测订单（与正常商品订单查询方式相同）；\n7. 在发货前消费者可以取消订单；\n8. 取消订单后3-5个工作日内退回款项；\n9. 一个账号参与众测活动的次数无限制，没有限购。每次只能购买一个商品，如需购买多个产品，可多次参与众测活动。']

In [23]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="summaries", embedding_function=embeddings)
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [24]:
summary_docs = [
    Document(page_content=s, metadata={id_key: doc_ids[i]})
    for i, s in enumerate(summaries)
]
summary_docs

[Document(page_content='华为商城推出了一项名为“0分期利息”的活动，允许用户使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息，由华为商城承担。从2023年开始，商城将相关宣传调整为“0分期利息”，以符合监管机构关于分期业务规范的要求。目前，该活动仅支持部分单品参与，且不能与其他商品一起购买。用户需要在订单提交成功后在支付方式页面选择分期支付并确认0分期利息的支付方式和期数后完成支付。需要注意的是，华为商城小程序暂不支持0分期利息。', metadata={'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'}),
 Document(page_content='华为商城众测活动是测试市场对新商品反应的一种低价试销形式，通过收集评价和销量来优化销售策略并引导商家改进。众测价通常优惠，不超过大促促销价原则，最终价格以与物权方谈判结果为准。所有产品都适用于华为商城的众测活动。\n\n参与方式：\n1. 在华为商城搜索“众测”进入入口；\n2. 众测频道每周一至周五不定期更新上架，部分商品可能会特别调整；\n3. 一款产品的众测时间通常为10-20天（热销的商品可能会延期5-10天）。\n\n常见问题：\n1. 众测商品主要来源是华为商城上架的新品和爆款商品回归上线；\n2. 众测商品价格对标618和双十一，一般是该商品某段时间的最低促销价；\n3. 众测商品质量与正式上架商品一致；\n4. 请以商品页显示为准；\n5. 提交订单后最长付款时效为24小时，逾期订单自动取消；\n6. 成功下单后可通过华为商城手机APP、华为商城手机WAP版、华为商城电脑网页版查询众测订单（与正常商品订单查询方式相同）；\n7. 在发货前消费者可以取消订单；\n8. 取消订单后3-5个工作日内退回款项；\n9. 一个账号参与众测活动的次数无限制，没有限购。每次只能购买一个商品，如需购买多个产品，可多次参与众测活动。', metadata={'doc_id': 'e744378a-08d3-4d3a-8a49-379da403b901'})]

In [25]:
retriever.vectorstore.add_documents(summary_docs)
retriever.docstore.mset(list(zip(doc_ids, docs)))

In [27]:
sub_docs = retriever.vectorstore.similarity_search("众测活动是否有参与限制？")
sub_docs

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(page_content='华为商城众测活动是测试市场对新商品反应的一种低价试销形式，通过收集评价和销量来优化销售策略并引导商家改进。众测价通常优惠，不超过大促促销价原则，最终价格以与物权方谈判结果为准。所有产品都适用于华为商城的众测活动。\n\n参与方式：\n1. 在华为商城搜索“众测”进入入口；\n2. 众测频道每周一至周五不定期更新上架，部分商品可能会特别调整；\n3. 一款产品的众测时间通常为10-20天（热销的商品可能会延期5-10天）。\n\n常见问题：\n1. 众测商品主要来源是华为商城上架的新品和爆款商品回归上线；\n2. 众测商品价格对标618和双十一，一般是该商品某段时间的最低促销价；\n3. 众测商品质量与正式上架商品一致；\n4. 请以商品页显示为准；\n5. 提交订单后最长付款时效为24小时，逾期订单自动取消；\n6. 成功下单后可通过华为商城手机APP、华为商城手机WAP版、华为商城电脑网页版查询众测订单（与正常商品订单查询方式相同）；\n7. 在发货前消费者可以取消订单；\n8. 取消订单后3-5个工作日内退回款项；\n9. 一个账号参与众测活动的次数无限制，没有限购。每次只能购买一个商品，如需购买多个产品，可多次参与众测活动。', metadata={'doc_id': 'e744378a-08d3-4d3a-8a49-379da403b901'}),
 Document(page_content='华为商城推出了一项名为“0分期利息”的活动，允许用户使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息，由华为商城承担。从2023年开始，商城将相关宣传调整为“0分期利息”，以符合监管机构关于分期业务规范的要求。目前，该活动仅支持部分单品参与，且不能与其他商品一起购买。用户需要在订单提交成功后在支付方式页面选择分期支付并确认0分期利息的支付方式和期数后完成支付。需要注意的是，华为商城小程序暂不支持0分期利息。', metadata={'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'})]

## 假设查询
LLM 还可用于生成针对特定文档可能提出的假设问题列表。然后可以嵌入这些问题


In [28]:
sub_docs = []
for i, doc in enumerate(docs):
    _id = doc_ids[i]
    _sub_docs = child_text_splitter.split_documents([doc])
    for _doc in _sub_docs:
        _doc.metadata[id_key] = _id
    sub_docs.extend(_sub_docs)
sub_docs

Created a chunk of size 224, which is longer than the specified 100
Created a chunk of size 117, which is longer than the specified 100


[Document(page_content='一、什么是0分期利息\n\n您好，“0分期利息”是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'}),
 Document(page_content='注：自2023年起，商城将相关宣传将“免息”调整为“0分期利息”，主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求“银行业金融机构应当在分期业务合同（协议）首页和业务办理页面以明显方式展示分期业务可能产生的所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。”', metadata={'source': './txt/faq-4359.txt', 'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'}),
 Document(page_content='二、可以参与0分期利息活动的商品\n\n商城目前仅支持部分单品参与0分期利息，若多商品（含不支持0分期利息）合并支付则不支持0分期利息，以支付页面为准，后续会逐渐开放更多商品，敬请关注。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'}),
 Document(page_content='三、确认订单分期成功\n\n订单提交成功后在支付方式页面选择分期支付，点选显示0分期利息的支付方式及具体0分期利息期数后，完成支付。', metadata={'source': './txt/faq-4359.txt', 'doc_id': '8a6959ea-ad9a-4df8-9181-232ea61dfaf7'}),
 Document(page_content='四、订单中有多个商品，其中有商品支持0分期利息，为什

In [31]:
from langchain_core.output_parsers import JsonOutputParser
promptStr = '''
```
{doc}
```

根据上面的文档，生成3个相关问题和回答。响应以json列表的结构返回。返回的结构参考如下
```
[
{{"question":"问题1","answer":"回答1"}},
{{"question":"问题2","answer":"回答2"}},
{{"question":"问题3","answer":"回答3"}}
]
```
'''

prompt = ChatPromptTemplate.from_template(promptStr)

chain = (
    {"doc": lambda x: x.page_content}
    | prompt
    | model
    | JsonOutputParser()
)

In [34]:
hypothetical_questions = chain.batch(sub_docs, {"max_concurrency": 5})

In [35]:
hypothetical_questions

[[{'question': '什么是0分期利息？',
   'answer': '0分期利息是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。'},
  {'question': '哪些渠道支持0分期利息？',
   'answer': '支持0分期利息的渠道包括花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物。'},
  {'question': '0分期利息由谁承担？', 'answer': '0分期利息由华为商城承担。'}],
 [{'question': "为什么商城将相关宣传从'免息'调整为'0分期利息'？",
   'answer': '主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求银行业金融机构在分期业务合同和业务办理页面以明显方式展示所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。'},
  {'question': '调整后的宣传方式有什么变化？',
   'answer': "调整后，商城将相关宣传从'免息'调整为'0分期利息'，主要变化是在展示分期业务收取的资金使用成本时，统一采用利息形式，并明确相应的计息规则。"},
  {'question': '调整后的宣传方式有什么优势？',
   'answer': '调整后的宣传方式更加清晰明了地展示了分期业务的资金使用成本和年化利率水平，有利于客户更清楚地了解相关费用和风险，保障客户的知情权和选择权。'}],
 [{'question': '哪些商品可以参与0分期利息活动？', 'answer': '目前仅支持部分单品参与0分期利息，后续会逐渐开放更多商品。'},
  {'question': '如果购买多件商品，其中一件不支持0分期利息，是否会影响其他商品的0分期利息资格？',
   'answer': '是的，若多商品（含不支持0分期利息）合并支付则不支持0分期利息。'},
  {'question': '如何查看哪些商品可以参与0分期利息活动？', 'answer': '在支付页面可以查看是否支持0分期利息。'}],
 [{

In [36]:
documents = []
for item in hypothetical_questions:
    for obj in item:
        content = "问：{}\n答：{}".format(obj['question'],obj['answer'])
        documents.append(Document(page_content=content))
documents

[Document(page_content='问：什么是0分期利息？\n答：0分期利息是指买家使用花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物时无需支付分期利息的功能，分期利息由华为商城承担。'),
 Document(page_content='问：哪些渠道支持0分期利息？\n答：支持0分期利息的渠道包括花呗、招行掌上生活、工行信用卡、银联信用卡等其他分期购物。'),
 Document(page_content='问：0分期利息由谁承担？\n答：0分期利息由华为商城承担。'),
 Document(page_content="问：为什么商城将相关宣传从'免息'调整为'0分期利息'？\n答：主要基于中国银保监会、中国人民银行《关于进一步促进信用卡业务规范健康发展的通知》（银保监规〔2022〕13号），要求银行业金融机构在分期业务合同和业务办理页面以明显方式展示所有息费项目、年化利率水平和息费计算方式。向客户展示分期业务收取的资金使用成本时，应当统一采用利息形式，并明确相应的计息规则，不得采用手续费等形式，法律法规另有规定的除外。"),
 Document(page_content="问：调整后的宣传方式有什么变化？\n答：调整后，商城将相关宣传从'免息'调整为'0分期利息'，主要变化是在展示分期业务收取的资金使用成本时，统一采用利息形式，并明确相应的计息规则。"),
 Document(page_content='问：调整后的宣传方式有什么优势？\n答：调整后的宣传方式更加清晰明了地展示了分期业务的资金使用成本和年化利率水平，有利于客户更清楚地了解相关费用和风险，保障客户的知情权和选择权。'),
 Document(page_content='问：哪些商品可以参与0分期利息活动？\n答：目前仅支持部分单品参与0分期利息，后续会逐渐开放更多商品。'),
 Document(page_content='问：如果购买多件商品，其中一件不支持0分期利息，是否会影响其他商品的0分期利息资格？\n答：是的，若多商品（含不支持0分期利息）合并支付则不支持0分期利息。'),
 Document(page_content='问：如何查看哪些商品可以参与0分期利息活动？\n答：在支付页面可以查看是否支持0分期利息。'),
 Document(page_content='

In [41]:
# The vectorstore to use to index the child chunks
vectorstore = Chroma(collection_name="Question", embedding_function=embeddings,persist_directory="./vector_store")
# The storage layer for the parent documents
store = InMemoryByteStore()
id_key = "doc_id"
# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorstore,
    byte_store=store,
    id_key=id_key,
)
doc_ids = [str(uuid.uuid4()) for _ in docs]

In [42]:
retriever.vectorstore.add_documents(documents)


['d174736c-dfdc-11ee-ad0c-e00af6642b58',
 'd174736d-dfdc-11ee-a4c9-e00af6642b58',
 'd174736e-dfdc-11ee-9963-e00af6642b58',
 'd174736f-dfdc-11ee-af3a-e00af6642b58',
 'd1747370-dfdc-11ee-8390-e00af6642b58',
 'd1747371-dfdc-11ee-8f18-e00af6642b58',
 'd1747372-dfdc-11ee-8431-e00af6642b58',
 'd1747373-dfdc-11ee-b079-e00af6642b58',
 'd1747374-dfdc-11ee-a9af-e00af6642b58',
 'd1747375-dfdc-11ee-824a-e00af6642b58',
 'd1747376-dfdc-11ee-9b3a-e00af6642b58',
 'd1747377-dfdc-11ee-86de-e00af6642b58',
 'd1747378-dfdc-11ee-8f1d-e00af6642b58',
 'd1747379-dfdc-11ee-9e24-e00af6642b58',
 'd174737a-dfdc-11ee-ae67-e00af6642b58',
 'd174737b-dfdc-11ee-878f-e00af6642b58',
 'd174737c-dfdc-11ee-8930-e00af6642b58',
 'd174737d-dfdc-11ee-b7f0-e00af6642b58',
 'd174737e-dfdc-11ee-af46-e00af6642b58',
 'd174737f-dfdc-11ee-92c5-e00af6642b58',
 'd1747380-dfdc-11ee-8c4d-e00af6642b58',
 'd1747381-dfdc-11ee-9cf7-e00af6642b58',
 'd1747382-dfdc-11ee-9185-e00af6642b58',
 'd1747383-dfdc-11ee-9e76-e00af6642b58',
 'd1747384-dfdc-

In [43]:
retriever.vectorstore.similarity_search("众测商品多久发货呢？")[0]

Document(page_content='问：众测商品买下后多久发货？\n答：您好，请以商品页显示为准。')